In [14]:
import pdal
import json
import geopandas as gpd
import pandas as pd

In [12]:
pipeline = [
        {
            "type": "readers.las",
            "filename": f'data/sample/MDS_color_3316-221.laz',
            "override_srs": "EPSG:31983"
        },
        {
            "type":"filters.range",
            "limits":"ReturnNumber[1:4],NumberOfReturns![1:1],UserData[3:],Intensity[:10],Classification[19:19]"
        },
        {
            "type":"filters.voxelcentroidnearestneighbor",
            "cell":0.50
        },
        {
            "type":"filters.outlier",
            "method":"radius",
            "radius":2.0,
            "min_k":4
        },
        {
            "type":"filters.range",
            "limits":"Classification![7:7]"
        },
        {
            "type":"writers.las",
            "filename": f'results/sample/MDS_3316-221_1000-filtered.laz'
        },
        {
            "type":"writers.text",
            "format":"csv",
            "order":"X,Y",
            "keep_unspecified":"false",
            "filename":"results/sample/MDS_3316-221_1000-filtered.csv"
        }
        
    ]

In [13]:
filtered = pdal.Pipeline(json.dumps(pipeline))
n_points = filtered.execute()

In [16]:
df_xy = pd.read_csv('results/sample/MDS_3316-221_1000-filtered.csv')

In [29]:
gdf_xy = gpd.GeoDataFrame(geometry=gpd.points_from_xy(x = df_xy.X, y = df_xy.Y))

In [34]:
gdf_xy.geometry = gdf_xy.buffer(2.5)

In [36]:
gdf_xy.dissolve().to_file('results/sample/buffer_dissolvido.gpkg', driver='GPKG')

In [4]:
# !docker rm postgis

In [5]:
# !docker run --name=postgis -d -e POSTGRES_USER=postgres -e POSTGRES_PASS=1234 -e POSTGRES_DBNAME=gis -e ALLOW_IP_RANGE=0.0.0.0/0 -p 5432:5432 -v pg_data:/var/lib/postgresql --restart=always kartoza/postgis

In [3]:
# !pip install sqlalchemy psycopg2 geoalchemy2